# Analysis of the quality of the data in the courses db

Due to how the data is gathered, there will likely be some gaps in the data. 
This should not significanly impact most analysis, but it is important to be aware of.

Assuming the data was collected at a regular interval, this analysis points out where gaps in the data might be found.

Sample causes of missing data:
- Data unavailable when requested from server
  - Server maintence
  - Network connectivity issue
- Unhandled changes in the path to the data
- Unhandled changes to the structure of the data
- Other unexpected error

In [1]:
import sqlite3
import pandas as pd
import datetime

dbname = "OMSCS_CA.db"
conn = sqlite3.connect(dbname)
# Tables not all guarenteed to have the same number of readings recorded
# ML used here as reference dataset as it's consistently a very popular course
ref_df = pd.read_sql_query("SELECT Timestamp FROM F18_87100", 
                           conn, 
                           parse_dates=['Timestamp']
                          )  # Machine Learning
# Create a table of the differences in timestamps between all table entries
diff_df = ref_df.diff()

In [2]:
# Simple gap determination if expected interval is known
timedelta = datetime.timedelta(minutes=35)
gaps_df = pd.DataFrame(columns=['Start','Stop','Diff'])
for row in diff_df.itertuples():
    if row[1] > timedelta:
        idx = row[0]
        gaps_df.loc[len(gaps_df)] = [ref_df.iloc[idx-1,0], ref_df.iloc[idx,0], row[1]]
print("Gaps identified:")
print(gaps_df)

Gaps identified:
                       Start                       Stop                   Diff
0 2018-08-04 04:41:39.716194 2018-08-05 11:42:12.237736 1 days 07:00:32.521542
1 2018-08-05 11:42:12.237736 2018-08-05 23:46:46.430161 0 days 12:04:34.192425
2 2018-08-06 15:46:33.348270 2018-08-06 16:46:33.135580 0 days 00:59:59.787310
3 2018-08-06 20:46:33.387863 2018-08-06 21:46:36.821216 0 days 01:00:03.433353
4 2018-08-07 08:16:32.512346 2018-08-07 09:01:34.565700 0 days 00:45:02.053354
5 2018-08-07 10:00:13.804891 2018-08-07 10:40:17.875072 0 days 00:40:04.070181
6 2018-08-08 03:09:28.824382 2018-08-08 04:09:34.142327 0 days 01:00:05.317945
7 2018-08-08 10:09:29.674694 2018-08-08 11:09:29.693565 0 days 01:00:00.018871
8 2018-08-08 19:39:29.946614 2018-08-08 20:39:30.063556 0 days 01:00:00.116942
9 2018-08-09 12:09:29.502975 2018-08-09 13:09:31.122096 0 days 01:00:01.619121


In [3]:
# Gap determination if expected gap is unknown

# Assuming the dataset has a constant interval, the list can be repeatedly 
# filtered until the variance is below some threshold
# However, if the dataset is very large and has high gap rate, a threshold would be needed to avoid too many iterations
# Also, a min expected gap could be considered to improve efficiency

# Remove outliers - Anything more than 1 std away from the mean
out_cutoff = (diff_df.mean(axis=0) + diff_df.std(axis=0))[0]
filtered_df = diff_df[diff_df['Timestamp'] < out_cutoff]

# Consider all points more than 1 std away from list with major outliers removed
# Might miss some gaps if the major outliers are very large
fmean = filtered_df.mean(axis=0)[0]
fstd = filtered_df.std(axis=0)[0]
out_cutoff = (fmean + fstd)
print("Gap identification characteristics:")
print(f"Gap if time diff > {out_cutoff}")
print(f"Mean:{fmean}")
print(f"Std: {fstd}\n")

# Create table of data gaps
gaps_df = pd.DataFrame(columns=['Start','Stop','Diff'])
for row in diff_df.itertuples():  
    if row[1] > out_cutoff:
        idx = row[0]
        gaps_df.loc[len(gaps_df)] = [ref_df.iloc[idx-1,0], ref_df.iloc[idx,0], row[1]]
print("Gaps identified:")
print(gaps_df) 

Gap identification characteristics:
Gap if time diff > 0 days 00:36:35.112266
Mean:0 days 00:30:54.436811
Std: 0 days 00:05:40.675454

Gaps identified:
                       Start                       Stop                   Diff
0 2018-08-04 04:41:39.716194 2018-08-05 11:42:12.237736 1 days 07:00:32.521542
1 2018-08-05 11:42:12.237736 2018-08-05 23:46:46.430161 0 days 12:04:34.192425
2 2018-08-06 15:46:33.348270 2018-08-06 16:46:33.135580 0 days 00:59:59.787310
3 2018-08-06 20:46:33.387863 2018-08-06 21:46:36.821216 0 days 01:00:03.433353
4 2018-08-07 08:16:32.512346 2018-08-07 09:01:34.565700 0 days 00:45:02.053354
5 2018-08-07 10:00:13.804891 2018-08-07 10:40:17.875072 0 days 00:40:04.070181
6 2018-08-08 03:09:28.824382 2018-08-08 04:09:34.142327 0 days 01:00:05.317945
7 2018-08-08 10:09:29.674694 2018-08-08 11:09:29.693565 0 days 01:00:00.018871
8 2018-08-08 19:39:29.946614 2018-08-08 20:39:30.063556 0 days 01:00:00.116942
9 2018-08-09 12:09:29.502975 2018-08-09 13:09:31.122096 0 